In [ ]:
# train 키워드 빈도수 n개 이상 남기기

In [24]:
import pandas as pd
from collections import Counter

In [36]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [37]:
train_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."


In [38]:
len(train_df.키워드[0])

330

In [39]:
def get_keyword_counts_by_class(df):
    """주어진 DataFrame에서 분류별로 키워드를 세고 반환합니다."""
    class_keyword_counts = {}
    
    for label in df['분류'].unique():
        class_df = df[df['분류'] == label]
        keywords = class_df['키워드'].str.split(',')
        all_keywords = [keyword for sublist in keywords for keyword in sublist]
        keyword_counts = Counter(all_keywords)
        class_keyword_counts[label] = keyword_counts
    
    return class_keyword_counts

In [40]:
# keyword_df = get_keyword_counts_by_class(train_df)
# keyword_df

In [41]:
def filter_keywords_by_class_count(df, min_count):
    """키워드가 min_count 이상인 것만 남기는 DataFrame을 반환합니다."""
    class_keyword_counts = get_keyword_counts_by_class(df)
    
    # 분류별로 3회 이하인 키워드 찾기
    low_count_keywords = {}
    for label, counts in class_keyword_counts.items():
        low_count_keywords[label] = {keyword for keyword, count in counts.items() if count <= min_count}
    
    # DataFrame의 키워드에서 각 분류별로 low_count_keywords를 제거합니다.
    def filter_keywords(row):
        keywords = row['키워드'].split(',')
        filtered_keywords = [keyword for keyword in keywords if keyword not in low_count_keywords[row['분류']]]
        return ','.join(filtered_keywords)

    # 필터링된 키워드로 업데이트
    df['키워드'] = df.apply(filter_keywords, axis=1)
    return df

In [42]:
filter_df = filter_keywords_by_class_count(train_df, min_count=3)
filter_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,대사..."


In [43]:
len(filter_df.키워드[0])

315